In [1]:
import os

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/3.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Vehicle Url'] != ''].reset_index(drop=True)
input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：10


['W19235',
 'W37425',
 'W37635',
 'W37872',
 'W37984',
 'W610150',
 'W610165',
 'W610166',
 'W72258',
 'W79985']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

df_temp, page = pd.DataFrame(), 0
while True:
    b = 0
    while True:
        b += 1
        print(b, page)
        try:
            request_url = f'''{input_.loc[a, 'Vehicle Url']}&start={page*50}&num=50'''
            
            resp = requests.get(request_url,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
        
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))
    
                if html.xpath('//div[@class="detail-app-pager"]'):
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
    if page == 0 and next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
        break
        
    # = = = = = = = = = = = = = = =

    list_th, list_tr = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')], html.xpath('//tr[@class="detail-app-row"]')

    # = = = = = = = = = = = = = = =

    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                 'Part Number': input_.loc[a, 'Part Number'],
                                 'Vehicle Url': input_.loc[a, 'Vehicle Url'],
                                 'Page': page + 1,
                                 'Row': [i+1 for i in range(len(list_tr))]})

    for i, tr in enumerate(list_tr):
        for j, th in enumerate(list_th):
            df_temp_temp.loc[i, th] = tr.xpath(f'./td[{j+1}]/text()')[0].strip()

    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    page += 1
    
    # = = = = = = = = = = = = = = =

    if not next_:
        break

df_temp

1 0


,No,Part Number,Vehicle Url,Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,1,1964,Studebaker,8E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
1,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,2,1964,Studebaker,8E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
2,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,3,1963,Studebaker,8E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
3,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,4,1963,Studebaker,8E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
4,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,5,1962,Studebaker,7E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
5,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,6,1962,Studebaker,7E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
6,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,7,1961,American Motors,Classic,,Rear,Bore: 13/16 In.
7,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,8,1961,Studebaker,6E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
8,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,9,1961,Studebaker,6E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
9,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,10,1960,American Motors,Rambler,,Rear,Bore: 13/16 In.


In [8]:
df_temp.empty

False

In [9]:
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,Vehicle Url,Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,1,1964,Studebaker,8E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
1,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,2,1964,Studebaker,8E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
2,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,3,1963,Studebaker,8E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
3,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,4,1963,Studebaker,8E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
4,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,5,1962,Studebaker,7E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
5,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,6,1962,Studebaker,7E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
6,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,7,1961,American Motors,Classic,,Rear,Bore: 13/16 In.
7,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,8,1961,Studebaker,6E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
8,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,9,1961,Studebaker,6E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
9,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,10,1960,American Motors,Rambler,,Rear,Bore: 13/16 In.


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct['Year'] = output_correct['Year'].astype(int)
    
    output_correct = output_correct.sort_values(by=['No', 'Page', 'Row'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Part Number,Vehicle Url,Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,1,1964,Studebaker,8E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
1,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,2,1964,Studebaker,8E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
2,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,3,1963,Studebaker,8E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
3,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,4,1963,Studebaker,8E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
4,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,5,1962,Studebaker,7E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
5,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,6,1962,Studebaker,7E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
6,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,7,1961,American Motors,Classic,,Rear,Bore: 13/16 In.
7,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,8,1961,Studebaker,6E10,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
8,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,9,1961,Studebaker,6E5,,Rear,Exc. Heavy Duty Brakes; Bore: 13/16 In.
9,1,W19235,https://www.dormanproducts.com/itemdetailapp.a...,1,10,1960,American Motors,Rambler,,Rear,Bore: 13/16 In.


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
